# Classifying Fashion-MNIST

Now it's your turn to build and train a neural network. You'll be using the [Fashion-MNIST dataset](https://github.com/zalandoresearch/fashion-mnist), a drop-in replacement for the MNIST dataset. MNIST is actually quite trivial with neural networks where you can easily achieve better than 97% accuracy. Fashion-MNIST is a set of 28x28 greyscale images of clothes. It's more complex than MNIST, so it's a better representation of the actual performance of your network, and a better representation of datasets you'll use in the real world.

<img src='assets/fashion-mnist-sprite.png' width=500px>

In this notebook, you'll build your own neural network. For the most part, you could just copy and paste the code from Part 3, but you wouldn't be learning. It's important for you to write the code yourself and get it to work. Feel free to consult the previous notebooks though as you work through this.

First off, let's load the dataset through torchvision.

In [10]:
import torch
from torchvision import datasets, transforms
import helper

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [11]:
print(trainset)
print('......train loader......',trainloader)
print(testset)
print('......test loader......',testloader)

Dataset FashionMNIST
    Number of datapoints: 60000
    Split: train
    Root Location: C:\Users\pavani/.pytorch/F_MNIST_data/
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.5,), std=(0.5,))
                         )
    Target Transforms (if any): None
......train loader...... <torch.utils.data.dataloader.DataLoader object at 0x000002A38F3230B8>
Dataset FashionMNIST
    Number of datapoints: 10000
    Split: test
    Root Location: C:\Users\pavani/.pytorch/F_MNIST_data/
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.5,), std=(0.5,))
                         )
    Target Transforms (if any): None
......test loader...... <torch.utils.data.dataloader.DataLoader object at 0x000002A38B525828>


Here we can see one of the images.

In [12]:
image, label = next(iter(trainloader))
#helper.imshow(image[0,:]);

In [13]:
print(image.shape)
print(label.shape)

torch.Size([64, 1, 28, 28])
torch.Size([64])


## Building the network

Here you should define your network. As with MNIST, each image is 28x28 which is a total of 784 pixels, and there are 10 classes. You should include at least one hidden layer. We suggest you use ReLU activations for the layers and to return the logits or log-softmax from the forward pass. It's up to you how many layers you add and the size of those layers.

In [14]:
# TODO: Define your network architecture here
from torch import nn, optim
import torch.nn.functional as F
class FMN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,64)
        self.fc4 = nn.Linear(64,10)
        
    def forward(self,x):  ##Forward Pass where we faltten images
        x = x.view(image.shape[0],-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x),dim=1) ##log softmax for calculating probabilities
        return x
    

In [15]:
FMN

__main__.FMN

# Train the network

Now you should create your network and train it. First you'll want to define [the criterion](http://pytorch.org/docs/master/nn.html#loss-functions) ( something like `nn.CrossEntropyLoss`) and [the optimizer](http://pytorch.org/docs/master/optim.html) (typically `optim.SGD` or `optim.Adam`).

Then write the training code. Remember the training pass is a fairly straightforward process:

* Make a forward pass through the network to get the logits 
* Use the logits to calculate the loss
* Perform a backward pass through the network with `loss.backward()` to calculate the gradients
* Take a step with the optimizer to update the weights

By adjusting the hyperparameters (hidden units, learning rate, etc), you should be able to get the training loss below 0.4.

In [16]:
# TODO: Create the network, define the criterion and optimizer

from torch import optim
model = FMN()
optimizer = optim.SGD(model.parameters(),lr=0.001)
#optimizer = optim.Adam(model.parameters(),lr=0.001)
criterion = nn.NLLLoss() #using Negative Log Softmax


In [17]:
# TODO: Train the network here
epoch = 10

for e in range(epoch):
    running_loss = 0   ###tracking loss
    for image,label in trainloader:
        logps = model(image)
        loss = criterion(logps,label)
        optimizer.zero_grad() ##zeroing grad
        loss.backward()  ##doing backward to find gradients
        optimizer.step()
        running_loss +=loss.item()
        
    else:
        print('Training Loss = ',running_loss)


Training Loss =  2123.5670232772827
Training Loss =  1968.6599674224854
Training Loss =  1585.3907486200333
Training Loss =  1211.5320217609406
Training Loss =  993.3601942062378
Training Loss =  858.8475688099861
Training Loss =  769.0249032974243
Training Loss =  710.0166330635548
Training Loss =  670.9963495731354
Training Loss =  642.5010378956795


In [19]:
#@As we see above at evry step our loss has minimized